In [291]:
try:
    spark.stop()
except Exception:
    pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Iceberg via REST")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.local.type", "rest")
    .config("spark.sql.catalog.local.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.local.warehouse", "s3://lake/warehouse")
    .config("spark.sql.catalog.local.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.local.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.local.s3.path-style-access", "true")
    .config("spark.sql.catalog.local.s3.access-key-id", "admin")
    .config("spark.sql.catalog.local.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.local.s3.region", "us-east-1")
    .getOrCreate()
)

spark

In [336]:
spark.sql("SHOW NAMESPACES IN local").show(truncate=False)

+---------+
|namespace|
+---------+
|raw      |
+---------+



In [337]:
spark.sql("SHOW TABLES IN local.raw").show(truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|raw      |avito    |false      |
+---------+---------+-----------+



In [338]:
tbl = "local.raw.avito"
spark.sql(f"SELECT COUNT(*) AS rows FROM {tbl}").show()


+----+
|rows|
+----+
| 130|
+----+



In [339]:
spark.sql("SELECT * FROM local.raw.avito LIMIT 1").show(truncate=False)

+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [340]:
# Load RAW table
raw_df = spark.table("local.raw.avito")   # or local.raw.sarouty

In [341]:
# Créer un DataFrame "Silver" minimal
silver_df = raw_df.select("id").distinct()

# Afficher quelques lignes
silver_df.show(5, truncate=False)

print("✅ Silver dataset initialized with only 'id' column.")
print("Total IDs:", silver_df.count())

+--------+
|id      |
+--------+
|56926340|
|57020711|
|56045169|
|56920617|
|57153823|
+--------+
only showing top 5 rows

✅ Silver dataset initialized with only 'id' column.
Total IDs: 104


In [342]:
from pyspark.sql import functions as F
raw_df.select("payload").first()["payload"]

'{"id": "56530450", "url": "https://www.avito.ma/fr/autre_secteur/bureaux/plateaux_Bureaux_100m²_AV_le_Caire_Temara_56530450.htm", "error": null, "title": "plateaux Bureaux 100m² AV le Caire Temara", "price_text": "8 500 DH", "breadcrumbs": "Home Icon Accueil > Tout le Maroc > Temara > Autre secteur > Avito Immobilier > Bureaux > plateaux Bureaux 100m² AV le Caire Temara", "category": "Bureaux, à louer", "description": "- AGENCE MOSTAGIMMO Met en location 2 Plateaux Bureaux 2 Façade ou 3em étage sans Ascenseur Superficie du 100m Sur Avenue le Caire. - Bureaux pour : Médecin, Architecte, paramédical, Comptable, Notaire. Pour plus d\'informations ou faire une Visite Contacter Votre Agence immobilière MOSTAGIMMO.", "attributes": "{\\"Salle de bain\\": \\"1\\", \\"Surface totale\\": \\"100\\", \\"Nombre de pièces\\": \\"1\\", \\"Étage\\": \\"3\\"}", "equipments": "1; 100; 1; 3; Câblage téléphonique", "seller_name": "AGENCE IMMOBILIER MOSTAGIMMO", "seller_type": "Particulier", "published_da

In [343]:
from pyspark.sql import functions as F, types as T

# 1) Définir le schéma du JSON dans "payload"
payload_schema = T.StructType([
    T.StructField("id", T.StringType()),
    T.StructField("url", T.StringType()),
    T.StructField("error", T.StringType()),
    T.StructField("title", T.StringType()),
    T.StructField("price_text", T.StringType()),
    T.StructField("breadcrumbs", T.StringType()),
    T.StructField("category", T.StringType()),
    T.StructField("description", T.StringType()),
    T.StructField("attributes", T.StringType()),  # JSON imbriqué sous forme de string
    T.StructField("equipments", T.StringType()),
    T.StructField("seller_name", T.StringType()),
    T.StructField("seller_type", T.StringType()),
    T.StructField("published_date", T.StringType()),
    T.StructField("image_urls", T.StringType()),
])

# 2) Parser le JSON depuis la colonne string "payload"
parsed = (raw_df
    .select(
        *[c for c in raw_df.columns if c != "payload"],  # ex: garder ingest_ts s'il existe
        F.from_json(F.col("payload"), payload_schema).alias("p")
    )
    .filter(F.col("p").isNotNull())  # ignorer les lignes avec JSON invalide
)

# 3) Parser le JSON imbriqué "attributes" -> Map<String,String>
attrs_map = F.from_json(F.col("p.attributes"), T.MapType(T.StringType(), T.StringType()))

# 4) Nettoyages utiles:
# - price_value (MAD) à partir de "price_text" (ex: "6 000 DH" -> 6000.0)
price_value = F.regexp_replace(F.col("p.price_text"), r"[^0-9]", "").cast("double")

# - image_urls -> array<string> en splittant sur " | " et trim de chaque url
image_urls_arr = F.transform(
    F.split(F.col("p.image_urls"), r"\s*\|\s*"),
    lambda x: F.trim(x)
)

# - equipments -> array<string> en splittant sur ";"
equipments_arr = F.transform(
    F.split(F.col("p.equipments"), r"\s*;\s*"),
    lambda x: F.trim(x)
)

# 5) Construire le DataFrame silver (colonnes à plat)
silver_df = parsed.select(
    F.col("p.id").alias("id"),
    F.col("p.url").alias("url"),
    F.col("p.title").alias("title"),
    F.col("p.price_text").alias("price_text"),
    price_value.alias("price_value_mad"),
    F.col("p.category").alias("category"),
    F.col("p.breadcrumbs").alias("breadcrumbs"),
    F.col("p.description").alias("description"),
    F.col("p.seller_name").alias("seller_name"),
    F.col("p.seller_type").alias("seller_type"),
    F.col("p.published_date").alias("published_date_text"),
    image_urls_arr.alias("image_urls"),
    equipments_arr.alias("equipments"),
    attrs_map.alias("attributes_map"),
    # garder le timestamp d'ingestion s'il est présent dans ton raw_df
    *([F.col("ingest_ts")] if "ingest_ts" in raw_df.columns else [])
)

In [344]:
import pandas as pd
from IPython.display import display

# Keep things compact
pd.set_option("display.max_columns", 20)   # don't try to show hundreds
pd.set_option("display.max_colwidth", 80)  # clamp long cells to ~80 chars

# pick a small sample and flatten newlines so rows stay short
pdf = (silver_df.limit(10)
       .toPandas()
       .replace({r'[\r\n\t]+': ' '}, regex=True))

# simple, compact table with ellipsis in long cells
display(
    pdf.style
      .set_table_styles([
          {'selector': 'table', 'props': [('table-layout','fixed'), ('width','100%')]},
          {'selector': 'th, td', 'props': [
              ('max-width','280px'),
              ('white-space','nowrap'),
              ('overflow','hidden'),
              ('text-overflow','ellipsis')
          ]}
      ])
      .hide(axis='index')  # remove row numbers
)
# show all distinct price_text values
silver_df.select("price_text").distinct().show(truncate=False)


id,url,title,price_text,price_value_mad,category,breadcrumbs,description,seller_name,seller_type,published_date_text,image_urls,equipments,attributes_map,ingest_ts
57020711,https://www.avito.ma/fr/lissasfa/bureaux/Bureau_à_louer_d_une_superficie_141_m²_57020711.htm,Bureau à louer d'une superficie 141 m²,12 690 DH,12690.000000,"Bureaux, à louer",Home Icon Accueil > Tout le Maroc > Casablanca > Lissasfa > Avito Immobilier > Bureaux > Bureau à louer d'une superficie 141 m²,"FBC met à votre disposition un plateau bureau 311 à louer de 141 m², situé dans un espace professionnel, sécurisé, calme et propre. Situé au 3ème étage, bien équipé : climatisation, 2 ascenseurs, cloisons, chauffage, parking et internet haut débit (fibre optique), kitchenette, accès direct à deux autoroutes, celle de rond-point Azbane et la Rocade sud-ouest. Pour plus d'information, n'hésitez pas à nous contacter.",FACILITIES BUSINESS CENTER,Particulier,2025-11-02 11:17:06,"['https://content.avito.ma/classifieds/images/10139961083?t=images', 'https://content.avito.ma/classifieds/images/10139961084?t=images', 'https://content.avito.ma/classifieds/images/10139961070?t=images', 'https://content.avito.ma/classifieds/images/10139961074?t=images', 'https://content.avito.ma/classifieds/images/10139961073?t=images', 'https://content.avito.ma/classifieds/images/10139961069?t=images', 'https://content.avito.ma/classifieds/images/10139961068?t=images', 'https://content.avito.ma/classifieds/images/10139961072?t=images', 'https://content.avito.ma/classifieds/images/10139961076?t=images', 'https://content.avito.ma/classifieds/images/10139961077?t=images', 'https://content.avito.ma/classifieds/images/10139961078?t=images', 'https://content.avito.ma/classifieds/images/10139961082?t=images']","['0', '141', '3', '3', 'Ascenseur', 'Chauffage', 'Climatisation', 'Câblage téléphonique', 'Parking', 'Sécurité']","{'Nombre de pièces': '3', 'Salle de bain': '0', 'Étage': '3', 'Surface totale': '141'}",2025-11-03 00:17:15.003000
57055336,https://www.avito.ma/fr/hay_riad/appartements/Appartement_en_location_à_Hay_Riad_Rabat_57055336.htm,Appartement en location à Hay Riad Rabat,None,nan,"Appartements, à louer",Home Icon Accueil > Tout le Maroc > Rabat > Hay Riad > Avito Immobilier > Locations Immobilières > Appartements > Appartement en location à Hay Riad Rabat,"Bel appartement de 195 m² en location au 1 er étage situé à Hay Riad Rabat Composé de deux salons avec cheminée séjour cuisine salle d'inviter deux chambres avec salle de bain suite parental avec salle de bain et drissing prix de location 16000dhs / mois pour plus d'informations veuillez contacter nous ,",Immosaadaoui,Particulier,2025-11-02 18:17:08,"['https://content.avito.ma/classifieds/images/10140311388?t=images', 'https://content.avito.ma/classifieds/images/10140311398?t=images', 'https://content.avito.ma/classifieds/images/10140311399?t=images', 'https://content.avito.ma/classifieds/images/10140311404?t=images', 'https://content.avito.ma/classifieds/images/10140311389?t=images', 'https://content.avito.ma/classifieds/images/10140311393?t=images', 'https://content.avito.ma/classifieds/images/10140311387?t=images']","['4', '195', '1']","{'Chambres': '4', 'Étage': '1', 'Surface habitable': '195'}",2025-11-03 00:17:15.003000
57140550,https://www.avito.ma/fr/targa/bureaux/Bureaux_privés_Domiciliation_à_Targa__Marrakech_57140550.htm,"Bureaux privés-Domiciliation à Targa, Marrakech",1 200 DH,1200.000000,"Bureaux, à louer","Home Icon Accueil > Tout le Maroc > Marrakech > Targa > Avito Immobilier > Bureaux > Bureaux privés-Domiciliation à Targa, Marrakech",🏢 Bureaux privés & domiciliation à Marrakech Besoin d’un espace professionnel clé en main à Marrakech ? Excellentia Business Center vous propose : ✅ Bureaux privés équipés et climatisés – prêts à l’emploi ✅ Internet fibre optique haut débit inclus ✅ Service de domiciliation 100 % en ligne – obtenez votre adresse professionnelle sans vous déplacer ✅ Réception et gestion de votre cou

+------------+
|price_text  |
+------------+
|35 000 DH   |
|1 200 DH    |
|6 500 DH    |
|560 000 DH  |
|25 000 DH   |
|6 300 DH    |
|9 880 DH    |
|10 000 DH   |
|3 200 000 DH|
|12 690 DH   |
|14 000 DH   |
|500 000 DH  |
|2 200 DH    |
|3 100 000 DH|
|5 250 DH    |
|2 500 DH    |
|3 700 DH    |
|7 900 DH    |
|2 500 000 DH|
|490 000 DH  |
+------------+
only showing top 20 rows



In [345]:
from pyspark.sql import functions as F

ID_COL = "id"  # change if your column is named differently
df = silver_df.withColumn("id_norm", F.trim(F.col(ID_COL).cast("string")))

In [346]:
total_rows    = df.count()
distinct_ids  = df.select("id_norm").distinct().count()
dup_row_count = total_rows - distinct_ids
print(f"Total rows = {total_rows} | Distinct IDs = {distinct_ids} | Duplicate rows (extra) = {dup_row_count}")

Total rows = 130 | Distinct IDs = 104 | Duplicate rows (extra) = 26


In [347]:
from pyspark.sql import functions as F

# (optionnel) filtrer les id vides
silver_df = silver_df.filter(F.col("id").isNotNull() & (F.trim(F.col("id")) != ""))

silver_df = (
    silver_df
      .withColumn("id_norm",
                  F.regexp_replace(F.trim(F.col("id").cast("string")), r"\s+", ""))
      .dropDuplicates(["id_norm"])
      .drop("id_norm")
)

In [348]:
total_rows    = df.count()
distinct_ids  = df.select("id_norm").distinct().count()
dup_row_count = total_rows - distinct_ids
print(f"Total rows = {total_rows} | Distinct IDs = {distinct_ids} | Duplicate rows (extra) = {dup_row_count}")

Total rows = 130 | Distinct IDs = 104 | Duplicate rows (extra) = 26


In [349]:
from pyspark.sql import functions as F

silver_df.select(
    F.when(F.col("price_text").isNull(), F.lit("NULL"))
     .otherwise(F.col("price_text"))
     .alias("price_text")
).show(10, truncate=False)


+----------+
|price_text|
+----------+
|195 000 DH|
|7 200 DH  |
|8 500 DH  |
|6 300 DH  |
|NULL      |
|260 000 DH|
|9 880 DH  |
|260 000 DH|
|7 000 DH  |
|11 000 DH |
+----------+
only showing top 10 rows



In [350]:
silver_df = (
    silver_df
      # Rename 'price_value_mad' to 'price_value'
      .withColumnRenamed("price_value_mad", "price_value")
      # Drop 'price_text' column
      .drop("price_text")
)

In [351]:
import pandas as pd
from IPython.display import display

# Keep things compact
pd.set_option("display.max_columns", 20)   # don't try to show hundreds
pd.set_option("display.max_colwidth", 80)  # clamp long cells to ~80 chars

# pick a small sample and flatten newlines so rows stay short
pdf = (silver_df.limit(10)
       .toPandas()
       .replace({r'[\r\n\t]+': ' '}, regex=True))

# simple, compact table with ellipsis in long cells
display(
    pdf.style
      .set_table_styles([
          {'selector': 'table', 'props': [('table-layout','fixed'), ('width','100%')]},
          {'selector': 'th, td', 'props': [
              ('max-width','280px'),
              ('white-space','nowrap'),
              ('overflow','hidden'),
              ('text-overflow','ellipsis')
          ]}
      ])
      .hide(axis='index')  # remove row numbers
)
# show all distinct price_text values


id,url,title,price_value,category,breadcrumbs,description,seller_name,seller_type,published_date_text,image_urls,equipments,attributes_map,ingest_ts
37619566,https://www.avito.ma/fr/autre_secteur/appartements/Appartement_Riad_Essalam_Beni_Yekhlef__Mohammedia_37619566.htm,Appartement Riad Essalam Beni Yekhlef- Mohammedia,195000.000000,"Appartements, à vendre",Home Icon Accueil > Tout le Maroc > Mohammedia > Autre secteur > Avito Immobilier > Ventes Immobilières > Appartements > Appartement Riad Essalam Beni Yekhlef- Mohammedia,Résidences économiques avec ascenseurs. Une Chambre/ salon/ cuisine/salle de bain. Prix : 195 000DH Livraison immédiate.,Groupe Ikamati,Particulier,2025-11-03 00:17:19,"['https://content.avito.ma/classifieds/images/2533557603?t=images', 'https://content.avito.ma/classifieds/images/2539801094?t=images', 'https://content.avito.ma/classifieds/images/2516510965?t=images', 'https://content.avito.ma/classifieds/images/2504023983?t=images', 'https://content.avito.ma/classifieds/images/2582417689?t=images', 'https://content.avito.ma/classifieds/images/2572806528?t=images']","['1', '1', 'Neuf', 'Immédiate', '0', 'Economique', 'Ascenseur']","{'Condition': 'Neuf', 'Disponibilité': 'Immédiate', 'Salle de bain': '1', 'Standing': 'Economique', 'Chambres': '1', 'Salons': '0'}",2025-11-02 23:30:30.003000
37621281,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_Sidi_Bernoussi_Résidences_Al_Badr_37621281.htm,Appartement Sidi Bernoussi Résidences Al Badr,7200.000000,"Appartements, à vendre",Home Icon Accueil > Tout le Maroc > Casablanca > Sidi Bernoussi > Avito Immobilier > Ventes Immobilières > Appartements > Appartement Sidi Bernoussi Résidences Al Badr,"Complexe résidentiel Al Badr est un projet moyen standing du Groupe Ikamati situé au meilleur quartier à Sidi Bernoussi proche de toutes les commodités nécessaires : Écoles privées et publiques, commerces de proximité, boulangeries, pâtisseries, cafés, pharmacies, hammam, plateaux de bureau,… Avec un accès direct à l’autoroute Casablanca –Rabat. Des appartements livrables dans l’immédiat avec des superficies qui varient entre 74m² et 96m². Dotés d’une finition irréprochable : cuisines équipées, carrelage de qualité premium, volets roulants motorisés, l’utilisation du bois de chêne,… Les résidences sont sécurisées 24h/24 et 7j/7j. A l’occasion de la saison d'été on vous offre une offre exclusive : le tout à 7200 dh m² au lieu de 8200dh/m². offre limitée et valable jusqu'au 31/08/2023. Pour plus d’informations, appelez nous ou visitez nos bureaux de vente ouverts toute la semaine et même les week-ends de 9h à 19h.",Groupe Ikamati,Particulier,2025-11-03 00:32:17,"['https://content.avito.ma/classifieds/images/9761138868?t=images', 'https://content.avito.ma/classifieds/images/9751423563?t=images', 'https://content.avito.ma/classifieds/images/9792952316?t=images', 'https://content.avito.ma/classifieds/images/9771942015?t=images', 'https://content.avito.ma/classifieds/images/9712278782?t=images']","['3', '100', 'Neuf', '49', 'Moins de 1 an', '0', 'Ascenseur', 'Balcon', 'Cuisine équipée', 'Parking', 'Sécurité', 'Terrasse']","{'Condition': 'Neuf', 'Étage': '0', 'Surface habitable': '49', 'Chambres': '3', 'Surface totale': '100', 'Âge du bien': 'Moins de 1 an'}",2025-11-02 23:40:30.004000
37621456,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_à_vendre_64_m²_à_Casablanca_37621456.htm,Appartement à vendre 64 m² à Casablanca,8500.000000,"Appartements, à vendre",Home Icon Accueil > Tout le Maroc > Casablanca > Sidi Bernoussi > Avito Immobilier > Ventes Immobilières > Appartements > Appartement à vendre 64 m² à Casablanca,"Résidence sécurisée avec caméras de surveillance, ascenseur, parking sous-sol, parabole collective, piscine et climatisation pré-installée. Avec une finition de bon standing, les appartements de résidences El Yassamine disposent de cuisines équipées, Bois de chêne naturel, Volets roulants motorisés, Douche italienne, Sanitaire haut de

In [352]:
silver_df.select("category").distinct().show(truncate=False)

+----------------------------+
|category                    |
+----------------------------+
|Autre Immobilier, à vendre  |
|Bureaux, à louer            |
|Local, à louer              |
|Appartements, à louer       |
|Local, à vendre             |
|Villas et Riads, à louer    |
|Bureaux, à vendre           |
|Terrains et fermes, à vendre|
|Maisons, à vendre           |
|Appartements, à vendre      |
|Villas et Riads, à vendre   |
|NULL                        |
+----------------------------+



In [353]:
from pyspark.sql import functions as F

silver_df = (
    silver_df
      # Create 'type' column for 'à louer' or 'à vendre'
      .withColumn(
          "type",
          F.when(F.col("category").contains("à louer"), "à louer")
           .when(F.col("category").contains("à vendre"), "à vendre")
           .otherwise(None)
      )
      # Create 'category_type' column with only the property type (e.g., Maisons, Appartements)
      .withColumn(
          "category_type",
          F.when(F.col("category").contains("à louer") | F.col("category").contains("à vendre"), 
                 F.split(F.col("category"), ",")[0])
           .otherwise(None)
      )
      # Drop the original 'category' column
      .drop("category")
)

# Check the result
silver_df.select("type", "category_type").show(30, truncate=False)


+--------+------------------+
|type    |category_type     |
+--------+------------------+
|à vendre|Appartements      |
|à vendre|Appartements      |
|à vendre|Appartements      |
|à louer |Appartements      |
|à vendre|Villas et Riads   |
|à vendre|Terrains et fermes|
|à louer |Bureaux           |
|à vendre|Terrains et fermes|
|à vendre|Appartements      |
|à louer |Appartements      |
|à vendre|Appartements      |
|à louer |Appartements      |
|à louer |Bureaux           |
|à vendre|Appartements      |
|à vendre|Maisons           |
|à vendre|Maisons           |
|à vendre|Villas et Riads   |
|à vendre|Bureaux           |
|à vendre|Appartements      |
|à vendre|Terrains et fermes|
|à louer |Bureaux           |
|à vendre|Autre Immobilier  |
|à vendre|Terrains et fermes|
|à louer |Appartements      |
|à vendre|Appartements      |
|à vendre|Appartements      |
|à vendre|Appartements      |
|à vendre|Maisons           |
|à vendre|Villas et Riads   |
|à louer |Bureaux           |
+--------+

In [354]:
import pandas as pd
from IPython.display import display

# Keep things compact
pd.set_option("display.max_columns", 20)   # don't try to show hundreds
pd.set_option("display.max_colwidth", 80)  # clamp long cells to ~80 chars

# pick a small sample and flatten newlines so rows stay short
pdf = (silver_df.limit(10)
       .toPandas()
       .replace({r'[\r\n\t]+': ' '}, regex=True))

# simple, compact table with ellipsis in long cells
display(
    pdf.style
      .set_table_styles([
          {'selector': 'table', 'props': [('table-layout','fixed'), ('width','100%')]},
          {'selector': 'th, td', 'props': [
              ('max-width','280px'),
              ('white-space','nowrap'),
              ('overflow','hidden'),
              ('text-overflow','ellipsis')
          ]}
      ])
      .hide(axis='index')  # remove row numbers
)

id,url,title,price_value,breadcrumbs,description,seller_name,seller_type,published_date_text,image_urls,equipments,attributes_map,ingest_ts,type,category_type
37619566,https://www.avito.ma/fr/autre_secteur/appartements/Appartement_Riad_Essalam_Beni_Yekhlef__Mohammedia_37619566.htm,Appartement Riad Essalam Beni Yekhlef- Mohammedia,195000.000000,Home Icon Accueil > Tout le Maroc > Mohammedia > Autre secteur > Avito Immobilier > Ventes Immobilières > Appartements > Appartement Riad Essalam Beni Yekhlef- Mohammedia,Résidences économiques avec ascenseurs. Une Chambre/ salon/ cuisine/salle de bain. Prix : 195 000DH Livraison immédiate.,Groupe Ikamati,Particulier,2025-11-03 00:17:19,"['https://content.avito.ma/classifieds/images/2533557603?t=images', 'https://content.avito.ma/classifieds/images/2539801094?t=images', 'https://content.avito.ma/classifieds/images/2516510965?t=images', 'https://content.avito.ma/classifieds/images/2504023983?t=images', 'https://content.avito.ma/classifieds/images/2582417689?t=images', 'https://content.avito.ma/classifieds/images/2572806528?t=images']","['1', '1', 'Neuf', 'Immédiate', '0', 'Economique', 'Ascenseur']","{'Condition': 'Neuf', 'Disponibilité': 'Immédiate', 'Salle de bain': '1', 'Standing': 'Economique', 'Chambres': '1', 'Salons': '0'}",2025-11-02 23:30:30.003000,à vendre,Appartements
37621281,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_Sidi_Bernoussi_Résidences_Al_Badr_37621281.htm,Appartement Sidi Bernoussi Résidences Al Badr,7200.000000,Home Icon Accueil > Tout le Maroc > Casablanca > Sidi Bernoussi > Avito Immobilier > Ventes Immobilières > Appartements > Appartement Sidi Bernoussi Résidences Al Badr,"Complexe résidentiel Al Badr est un projet moyen standing du Groupe Ikamati situé au meilleur quartier à Sidi Bernoussi proche de toutes les commodités nécessaires : Écoles privées et publiques, commerces de proximité, boulangeries, pâtisseries, cafés, pharmacies, hammam, plateaux de bureau,… Avec un accès direct à l’autoroute Casablanca –Rabat. Des appartements livrables dans l’immédiat avec des superficies qui varient entre 74m² et 96m². Dotés d’une finition irréprochable : cuisines équipées, carrelage de qualité premium, volets roulants motorisés, l’utilisation du bois de chêne,… Les résidences sont sécurisées 24h/24 et 7j/7j. A l’occasion de la saison d'été on vous offre une offre exclusive : le tout à 7200 dh m² au lieu de 8200dh/m². offre limitée et valable jusqu'au 31/08/2023. Pour plus d’informations, appelez nous ou visitez nos bureaux de vente ouverts toute la semaine et même les week-ends de 9h à 19h.",Groupe Ikamati,Particulier,2025-11-03 00:32:17,"['https://content.avito.ma/classifieds/images/9761138868?t=images', 'https://content.avito.ma/classifieds/images/9751423563?t=images', 'https://content.avito.ma/classifieds/images/9792952316?t=images', 'https://content.avito.ma/classifieds/images/9771942015?t=images', 'https://content.avito.ma/classifieds/images/9712278782?t=images']","['3', '100', 'Neuf', '49', 'Moins de 1 an', '0', 'Ascenseur', 'Balcon', 'Cuisine équipée', 'Parking', 'Sécurité', 'Terrasse']","{'Condition': 'Neuf', 'Étage': '0', 'Surface habitable': '49', 'Chambres': '3', 'Surface totale': '100', 'Âge du bien': 'Moins de 1 an'}",2025-11-02 23:40:30.004000,à vendre,Appartements
37621456,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_à_vendre_64_m²_à_Casablanca_37621456.htm,Appartement à vendre 64 m² à Casablanca,8500.000000,Home Icon Accueil > Tout le Maroc > Casablanca > Sidi Bernoussi > Avito Immobilier > Ventes Immobilières > Appartements > Appartement à vendre 64 m² à Casablanca,"Résidence sécurisée avec caméras de surveillance, ascenseur, parking sous-sol, parabole collective, piscine et climatisation pré-installée. Avec une finition de bon standing, les appartements de résidences El Yassamine disposent de cuisines équipées, Bois de chêne naturel, Volets roulants motorisés, Douche italienne, Sanitaire haut de gamme, … La résidenc

In [355]:
silver_df.select("breadcrumbs").distinct().show(5 ,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|breadcrumbs                                                                                                                                                         |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Home Icon Accueil > Tout le Maroc > Temara > Autre secteur > Avito Immobilier > Local > Magasin 152m² Total Stock ou Show Rom Seulement                             |
|Home Icon Accueil > Tout le Maroc > Rabat > Hay Riad > Avito Immobilier > Locations Immobilières > Appartements > Appartement en location à Hay Riad Rabat          |
|Home Icon Accueil > Tout le Maroc > Casablanca > 2 Mars > Avito Immobilier > Local > 93, bd d'Anoual BEAUX LOCAUX Zone Très ANIMÉE                                  

In [356]:
from pyspark.sql import functions as F

# Split 'breadcrumbs' based on '>'
split_breadcrumbs = F.split(F.col("breadcrumbs"), " > ")

# Create new columns for each segment
silver_df = (
    silver_df
    .withColumn("city", split_breadcrumbs.getItem(2))  # e.g., "Casablanca"
    .withColumn("neighborhood", split_breadcrumbs.getItem(3))  # e.g., "Maarif"
    .withColumn("site", split_breadcrumbs.getItem(4))  # e.g., "Avito Immobilier"
    .drop("breadcrumbs")  # Drop the original column if not needed
)

# Check the result
silver_df.select(
     "city", "neighborhood", "site", 
).show(30, truncate=False)


+----------------------+-------------------+----------------+
|city                  |neighborhood       |site            |
+----------------------+-------------------+----------------+
|Mohammedia            |Autre secteur      |Avito Immobilier|
|Casablanca            |Sidi Bernoussi     |Avito Immobilier|
|Casablanca            |Sidi Bernoussi     |Avito Immobilier|
|Casablanca            |Riviera            |Avito Immobilier|
|Agadir                |Tamraght           |Avito Immobilier|
|Kénitra               |Autre secteur      |Avito Immobilier|
|Casablanca            |Roches Noires      |Avito Immobilier|
|Sidi Allal El Bahraoui|Toute la ville     |Avito Immobilier|
|Casablanca            |Sidi Bernoussi     |Avito Immobilier|
|Rabat                 |Aviation - Mabella |Avito Immobilier|
|Fès                   |Route de Sefrou    |Avito Immobilier|
|Nouaceur              |Toute la ville     |Avito Immobilier|
|Mohammedia            |Quartier du Parc   |Avito Immobilier|
|Casabla

In [357]:
from IPython.display import display

# Assuming 'silver_df' is already created as per the earlier transformation
display(
    silver_df.select(
        "city", "neighborhood", "site"
    ).toPandas()
)

,city,neighborhood,site
0,Mohammedia,Autre secteur,Avito Immobilier
1,Casablanca,Sidi Bernoussi,Avito Immobilier
2,Casablanca,Sidi Bernoussi,Avito Immobilier
3,Casablanca,Riviera,Avito Immobilier
4,Agadir,Tamraght,Avito Immobilier
...,...,...,...
99,El Jadida,Autre secteur,Avito Immobilier
100,El Jadida,Saada,Avito Immobilier
101,Salé,Sala El Jadida,Avito Immobilier
102,Skhirat,Autre secteur,Avito Immobilier


In [358]:
import pandas as pd
from IPython.display import display

# Keep things compact
pd.set_option("display.max_columns", 20)   # don't try to show hundreds
pd.set_option("display.max_colwidth", 80)  # clamp long cells to ~80 chars

# pick a small sample and flatten newlines so rows stay short
pdf = (silver_df.limit(10)
       .toPandas()
       .replace({r'[\r\n\t]+': ' '}, regex=True))

# simple, compact table with ellipsis in long cells
display(
    pdf.style
      .set_table_styles([
          {'selector': 'table', 'props': [('table-layout','fixed'), ('width','100%')]},
          {'selector': 'th, td', 'props': [
              ('max-width','280px'),
              ('white-space','nowrap'),
              ('overflow','hidden'),
              ('text-overflow','ellipsis')
          ]}
      ])
      .hide(axis='index')  # remove row numbers
)

id,url,title,price_value,description,seller_name,seller_type,published_date_text,image_urls,equipments,attributes_map,ingest_ts,type,category_type,city,neighborhood,site
37619566,https://www.avito.ma/fr/autre_secteur/appartements/Appartement_Riad_Essalam_Beni_Yekhlef__Mohammedia_37619566.htm,Appartement Riad Essalam Beni Yekhlef- Mohammedia,195000.000000,Résidences économiques avec ascenseurs. Une Chambre/ salon/ cuisine/salle de bain. Prix : 195 000DH Livraison immédiate.,Groupe Ikamati,Particulier,2025-11-03 00:17:19,"['https://content.avito.ma/classifieds/images/2533557603?t=images', 'https://content.avito.ma/classifieds/images/2539801094?t=images', 'https://content.avito.ma/classifieds/images/2516510965?t=images', 'https://content.avito.ma/classifieds/images/2504023983?t=images', 'https://content.avito.ma/classifieds/images/2582417689?t=images', 'https://content.avito.ma/classifieds/images/2572806528?t=images']","['1', '1', 'Neuf', 'Immédiate', '0', 'Economique', 'Ascenseur']","{'Condition': 'Neuf', 'Disponibilité': 'Immédiate', 'Salle de bain': '1', 'Standing': 'Economique', 'Chambres': '1', 'Salons': '0'}",2025-11-02 23:30:30.003000,à vendre,Appartements,Mohammedia,Autre secteur,Avito Immobilier
37621281,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_Sidi_Bernoussi_Résidences_Al_Badr_37621281.htm,Appartement Sidi Bernoussi Résidences Al Badr,7200.000000,"Complexe résidentiel Al Badr est un projet moyen standing du Groupe Ikamati situé au meilleur quartier à Sidi Bernoussi proche de toutes les commodités nécessaires : Écoles privées et publiques, commerces de proximité, boulangeries, pâtisseries, cafés, pharmacies, hammam, plateaux de bureau,… Avec un accès direct à l’autoroute Casablanca –Rabat. Des appartements livrables dans l’immédiat avec des superficies qui varient entre 74m² et 96m². Dotés d’une finition irréprochable : cuisines équipées, carrelage de qualité premium, volets roulants motorisés, l’utilisation du bois de chêne,… Les résidences sont sécurisées 24h/24 et 7j/7j. A l’occasion de la saison d'été on vous offre une offre exclusive : le tout à 7200 dh m² au lieu de 8200dh/m². offre limitée et valable jusqu'au 31/08/2023. Pour plus d’informations, appelez nous ou visitez nos bureaux de vente ouverts toute la semaine et même les week-ends de 9h à 19h.",Groupe Ikamati,Particulier,2025-11-03 00:32:17,"['https://content.avito.ma/classifieds/images/9761138868?t=images', 'https://content.avito.ma/classifieds/images/9751423563?t=images', 'https://content.avito.ma/classifieds/images/9792952316?t=images', 'https://content.avito.ma/classifieds/images/9771942015?t=images', 'https://content.avito.ma/classifieds/images/9712278782?t=images']","['3', '100', 'Neuf', '49', 'Moins de 1 an', '0', 'Ascenseur', 'Balcon', 'Cuisine équipée', 'Parking', 'Sécurité', 'Terrasse']","{'Condition': 'Neuf', 'Étage': '0', 'Surface habitable': '49', 'Chambres': '3', 'Surface totale': '100', 'Âge du bien': 'Moins de 1 an'}",2025-11-02 23:40:30.004000,à vendre,Appartements,Casablanca,Sidi Bernoussi,Avito Immobilier
37621456,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_à_vendre_64_m²_à_Casablanca_37621456.htm,Appartement à vendre 64 m² à Casablanca,8500.000000,"Résidence sécurisée avec caméras de surveillance, ascenseur, parking sous-sol, parabole collective, piscine et climatisation pré-installée. Avec une finition de bon standing, les appartements de résidences El Yassamine disposent de cuisines équipées, Bois de chêne naturel, Volets roulants motorisés, Douche italienne, Sanitaire haut de gamme, … La résidence est à proximité de : Écoles privée et publique, commerces de proximité, boulangeries, pâtisseries, cafés, pharmacies, hammam, plateaux de bureau. Réservez votre appartement et bénéficier de remises exceptionnelles sur le prix de vente.",Groupe Ikamati,Particulier,2025-11-03 00:21:11,"['https://content.avito.ma/classifieds/images/9757591781?t=images', 'https://content.avito.ma/classifieds/images/77422

In [359]:
silver_df.select("equipments").distinct().show(5 , truncate=False)

+-----------------------------------------------------------------------------------------------------+
|equipments                                                                                           |
+-----------------------------------------------------------------------------------------------------+
|[2, 2, 150, 1 mois, 250, 1, Concierge, Sécurité]                                                     |
|[2, 2, 86, 1 mois, 200, 1, Ascenseur, Balcon, Concierge, Cuisine équipée, Parking, Sécurité]         |
|[1, 1, 45, 1 mois, 1, 45, Ascenseur, Balcon, Climatisation, Concierge, Cuisine équipée, Meublé]      |
|[3, 100, Neuf, 49, Moins de 1 an, 0, Ascenseur, Balcon, Cuisine équipée, Parking, Sécurité, Terrasse]|
|[1, 100, 1, 3, Câblage téléphonique]                                                                 |
+-----------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [360]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
import re

# Define a UDF to clean the 'equipments' list, handling NoneType
def clean_equipments(equipments):
    # Handle NoneType case
    if equipments is None:
        return []
    
    # Filter out unwanted entries (numbers and time-related strings)
    cleaned = [item for item in equipments if not re.match(r'^\d+$', str(item)) and not re.match(r'.*(mois|an).*', str(item))]
    return cleaned

# Register the UDF
clean_equipments_udf = udf(clean_equipments, ArrayType(StringType()))

# Apply the UDF to clean 'equipments' directly (in place)
silver_df = silver_df.withColumn("equipments", clean_equipments_udf(col("equipments")))

# Show the cleaned 'equipments' column
silver_df.select("equipments").show(5, truncate=False)

+-----------------------------------------------------------------------+
|equipments                                                             |
+-----------------------------------------------------------------------+
|[Neuf, Immédiate, Economique, Ascenseur]                               |
|[Neuf, Ascenseur, Balcon, Cuisine équipée, Parking, Sécurité, Terrasse]|
|[]                                                                     |
|[Ascenseur, Balcon, Climatisation, Concierge, Cuisine équipée, Meublé] |
|[Balcon, Chauffage, Climatisation, Cuisine équipée, Garage, Meublé]    |
+-----------------------------------------------------------------------+
only showing top 5 rows



In [361]:
from pyspark.sql import functions as F

# Extract the keys and values from the attributes_map
keys = silver_df.select(F.explode(F.map_keys(F.col("attributes_map"))).alias("attribute")).distinct().rdd.flatMap(lambda x: x).collect()

# For each key, create a new column with its corresponding value
for key in keys:
    silver_df = silver_df.withColumn(
        key, 
        F.when(F.col("attributes_map").getItem(key).isNotNull(), 
               F.col("attributes_map").getItem(key)).otherwise(None)
    )

# Show the result with new columns for each attribute
silver_df.select("id", *keys).show(5, truncate=False)

+--------+-----------------+-------+------+------------------+----------+--------------+-----+-------------+--------+-------------+----------------+----------------------+---------+--------------+-------------+------+
|id      |Surface habitable|Caution|Zoning|Type d'appartement|Standing  |Surface totale|Étage|Âge du bien  |Chambres|Salle de bain|Nombre de pièces|Frais de syndic / mois|Condition|Nombre d'étage|Disponibilité|Salons|
+--------+-----------------+-------+------+------------------+----------+--------------+-----+-------------+--------+-------------+----------------+----------------------+---------+--------------+-------------+------+
|37619566|NULL             |NULL   |NULL  |NULL              |Economique|NULL          |NULL |NULL         |1       |1            |NULL            |NULL                  |Neuf     |NULL          |Immédiate    |0     |
|37621281|49               |NULL   |NULL  |NULL              |NULL      |100           |0    |Moins de 1 an|3       |NULL       

In [379]:
import pandas as pd
from IPython.display import display

# Keep things compact
pd.set_option("display.max_columns", 20)   # don't try to show hundreds
pd.set_option("display.max_colwidth", 80)  # clamp long cells to ~80 chars

# pick a small sample and flatten newlines so rows stay short
pdf = (silver_df.limit(10)
       .toPandas()
       .replace({r'[\r\n\t]+': ' '}, regex=True))

# simple, compact table with ellipsis in long cells
display(
    pdf.style
      .set_table_styles([
          {'selector': 'table', 'props': [('table-layout','fixed'), ('width','100%')]},
          {'selector': 'th, td', 'props': [
              ('max-width','280px'),
              ('white-space','nowrap'),
              ('overflow','hidden'),
              ('text-overflow','ellipsis')
          ]}
      ])
      .hide(axis='index')  # remove row numbers
)

id,url,title,price_value,description,seller_name,seller_type,published_date_text,image_urls,equipments,ingest_ts,type,category_type,city,neighborhood,site,Surface habitable,Caution,Zoning,Type d'appartement,Standing,Surface totale,Étage,Âge du bien,Chambres,Salle de bain,Nombre de pièces,Frais de syndic / mois,Condition,Nombre d'étage,Disponibilité,Salons
37619566,https://www.avito.ma/fr/autre_secteur/appartements/Appartement_Riad_Essalam_Beni_Yekhlef__Mohammedia_37619566.htm,Appartement Riad Essalam Beni Yekhlef- Mohammedia,195000.000000,Résidences économiques avec ascenseurs. Une Chambre/ salon/ cuisine/salle de bain. Prix : 195 000DH Livraison immédiate.,Groupe Ikamati,Particulier,2025-11-03 00:17:19,"['https://content.avito.ma/classifieds/images/2533557603?t=images', 'https://content.avito.ma/classifieds/images/2539801094?t=images', 'https://content.avito.ma/classifieds/images/2516510965?t=images', 'https://content.avito.ma/classifieds/images/2504023983?t=images', 'https://content.avito.ma/classifieds/images/2582417689?t=images', 'https://content.avito.ma/classifieds/images/2572806528?t=images']","['Neuf', 'Immédiate', 'Economique', 'Ascenseur']",2025-11-02 23:30:30.003000,sale,Appartements,Mohammedia,Autre secteur,avito,None,None,None,None,Economique,None,None,None,1,1,None,None,Neuf,None,Immédiate,0
37621281,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_Sidi_Bernoussi_Résidences_Al_Badr_37621281.htm,Appartement Sidi Bernoussi Résidences Al Badr,7200.000000,"Complexe résidentiel Al Badr est un projet moyen standing du Groupe Ikamati situé au meilleur quartier à Sidi Bernoussi proche de toutes les commodités nécessaires : Écoles privées et publiques, commerces de proximité, boulangeries, pâtisseries, cafés, pharmacies, hammam, plateaux de bureau,… Avec un accès direct à l’autoroute Casablanca –Rabat. Des appartements livrables dans l’immédiat avec des superficies qui varient entre 74m² et 96m². Dotés d’une finition irréprochable : cuisines équipées, carrelage de qualité premium, volets roulants motorisés, l’utilisation du bois de chêne,… Les résidences sont sécurisées 24h/24 et 7j/7j. A l’occasion de la saison d'été on vous offre une offre exclusive : le tout à 7200 dh m² au lieu de 8200dh/m². offre limitée et valable jusqu'au 31/08/2023. Pour plus d’informations, appelez nous ou visitez nos bureaux de vente ouverts toute la semaine et même les week-ends de 9h à 19h.",Groupe Ikamati,Particulier,2025-11-03 00:32:17,"['https://content.avito.ma/classifieds/images/9761138868?t=images', 'https://content.avito.ma/classifieds/images/9751423563?t=images', 'https://content.avito.ma/classifieds/images/9792952316?t=images', 'https://content.avito.ma/classifieds/images/9771942015?t=images', 'https://content.avito.ma/classifieds/images/9712278782?t=images']","['Neuf', 'Ascenseur', 'Balcon', 'Cuisine équipée', 'Parking', 'Sécurité', 'Terrasse']",2025-11-02 23:40:30.004000,sale,Appartements,Casablanca,Sidi Bernoussi,avito,49,None,None,None,None,100,0,Moins de 1 an,3,None,None,None,Neuf,None,None,None
37621456,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_à_vendre_64_m²_à_Casablanca_37621456.htm,Appartement à vendre 64 m² à Casablanca,8500.000000,"Résidence sécurisée avec caméras de surveillance, ascenseur, parking sous-sol, parabole collective, piscine et climatisation pré-installée. Avec une finition de bon standing, les appartements de résidences El Yassamine disposent de cuisines équipées, Bois de chêne naturel, Volets roulants motorisés, Douche italienne, Sanitaire haut de gamme, … La résidence est à proximité de : Écoles privée et publique, commerces de proximité, boulangeries, pâtisseries, cafés, pharmacies, hammam, plateaux de bureau. Réservez votre appartement et bénéficier de remises exceptionnelles sur le prix de vente.",Groupe Ikamati,Particulier,2025-11-03 00:21:11,"['https://content.avito.ma/classifieds/images/9757591781?t=images', 'https://content.avito.ma/classifieds/images/7742218029?t=

In [363]:
# Get the number of columns
num_columns = len(silver_df.columns)

# Display the result
print(f"Number of columns in the dataframe: {num_columns}")

Number of columns in the dataframe: 33


In [364]:
from pyspark.sql import functions as F

# Replace all values in the 'site' column with "avito"
silver_df = silver_df.withColumn("site", F.lit("avito"))

# Show the updated DataFrame (distinct 'site' values)
silver_df.select("site").distinct().show(5, truncate=False)

+-----+
|site |
+-----+
|avito|
+-----+



In [365]:
from pyspark.sql import functions as F

# Update the 'type' column to have English values
silver_df = silver_df.withColumn(
    "type", 
    F.when(F.col("type") == "à louer", "rent")
     .when(F.col("type") == "à vendre", "sale")
     .otherwise(F.col("type"))
)

# Show the result with counts for the updated 'type' column
type_counts_english = silver_df.groupBy("type").count()
type_counts_english.show(truncate=False)

+----+-----+
|type|count|
+----+-----+
|NULL|2    |
|sale|58   |
|rent|44   |
+----+-----+



In [366]:
# Group by 'seller_type' and count the occurrences
seller_type_counts = silver_df.groupBy("seller_type").count()

# Show the result
seller_type_counts.show(truncate=False)

+-----------+-----+
|seller_type|count|
+-----------+-----+
|NULL       |1    |
|Boutique   |1    |
|Particulier|102  |
+-----------+-----+



In [376]:
import pandas as pd
from IPython.display import display

# Keep things compact
pd.set_option("display.max_columns", 20)   # don't try to show hundreds
pd.set_option("display.max_colwidth", 80)  # clamp long cells to ~80 chars

# pick a small sample and flatten newlines so rows stay short
pdf = (silver_df.limit(10)
       .toPandas()
       .replace({r'[\r\n\t]+': ' '}, regex=True))

# simple, compact table with ellipsis in long cells
display(
    pdf.style
      .set_table_styles([
          {'selector': 'table', 'props': [('table-layout','fixed'), ('width','100%')]},
          {'selector': 'th, td', 'props': [
              ('max-width','280px'),
              ('white-space','nowrap'),
              ('overflow','hidden'),
              ('text-overflow','ellipsis')
          ]}
      ])
      .hide(axis='index')  # remove row numbers
)

id,url,title,price_value,description,seller_name,seller_type,published_date_text,image_urls,equipments,ingest_ts,type,category_type,city,neighborhood,site,Surface habitable,Caution,Zoning,Type d'appartement,Standing,Surface totale,Étage,Âge du bien,Chambres,Salle de bain,Nombre de pièces,Frais de syndic / mois,Condition,Nombre d'étage,Disponibilité,Salons
37619566,https://www.avito.ma/fr/autre_secteur/appartements/Appartement_Riad_Essalam_Beni_Yekhlef__Mohammedia_37619566.htm,Appartement Riad Essalam Beni Yekhlef- Mohammedia,195000.000000,Résidences économiques avec ascenseurs. Une Chambre/ salon/ cuisine/salle de bain. Prix : 195 000DH Livraison immédiate.,Groupe Ikamati,Particulier,2025-11-03 00:17:19,"['https://content.avito.ma/classifieds/images/2533557603?t=images', 'https://content.avito.ma/classifieds/images/2539801094?t=images', 'https://content.avito.ma/classifieds/images/2516510965?t=images', 'https://content.avito.ma/classifieds/images/2504023983?t=images', 'https://content.avito.ma/classifieds/images/2582417689?t=images', 'https://content.avito.ma/classifieds/images/2572806528?t=images']","['Neuf', 'Immédiate', 'Economique', 'Ascenseur']",2025-11-02 23:30:30.003000,sale,Appartements,Mohammedia,Autre secteur,avito,None,None,None,None,Economique,None,None,None,1,1,None,None,Neuf,None,Immédiate,0
37621281,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_Sidi_Bernoussi_Résidences_Al_Badr_37621281.htm,Appartement Sidi Bernoussi Résidences Al Badr,7200.000000,"Complexe résidentiel Al Badr est un projet moyen standing du Groupe Ikamati situé au meilleur quartier à Sidi Bernoussi proche de toutes les commodités nécessaires : Écoles privées et publiques, commerces de proximité, boulangeries, pâtisseries, cafés, pharmacies, hammam, plateaux de bureau,… Avec un accès direct à l’autoroute Casablanca –Rabat. Des appartements livrables dans l’immédiat avec des superficies qui varient entre 74m² et 96m². Dotés d’une finition irréprochable : cuisines équipées, carrelage de qualité premium, volets roulants motorisés, l’utilisation du bois de chêne,… Les résidences sont sécurisées 24h/24 et 7j/7j. A l’occasion de la saison d'été on vous offre une offre exclusive : le tout à 7200 dh m² au lieu de 8200dh/m². offre limitée et valable jusqu'au 31/08/2023. Pour plus d’informations, appelez nous ou visitez nos bureaux de vente ouverts toute la semaine et même les week-ends de 9h à 19h.",Groupe Ikamati,Particulier,2025-11-03 00:32:17,"['https://content.avito.ma/classifieds/images/9761138868?t=images', 'https://content.avito.ma/classifieds/images/9751423563?t=images', 'https://content.avito.ma/classifieds/images/9792952316?t=images', 'https://content.avito.ma/classifieds/images/9771942015?t=images', 'https://content.avito.ma/classifieds/images/9712278782?t=images']","['Neuf', 'Ascenseur', 'Balcon', 'Cuisine équipée', 'Parking', 'Sécurité', 'Terrasse']",2025-11-02 23:40:30.004000,sale,Appartements,Casablanca,Sidi Bernoussi,avito,49,None,None,None,None,100,0,Moins de 1 an,3,None,None,None,Neuf,None,None,None
37621456,https://www.avito.ma/fr/sidi_bernoussi/appartements/Appartement_à_vendre_64_m²_à_Casablanca_37621456.htm,Appartement à vendre 64 m² à Casablanca,8500.000000,"Résidence sécurisée avec caméras de surveillance, ascenseur, parking sous-sol, parabole collective, piscine et climatisation pré-installée. Avec une finition de bon standing, les appartements de résidences El Yassamine disposent de cuisines équipées, Bois de chêne naturel, Volets roulants motorisés, Douche italienne, Sanitaire haut de gamme, … La résidence est à proximité de : Écoles privée et publique, commerces de proximité, boulangeries, pâtisseries, cafés, pharmacies, hammam, plateaux de bureau. Réservez votre appartement et bénéficier de remises exceptionnelles sur le prix de vente.",Groupe Ikamati,Particulier,2025-11-03 00:21:11,"['https://content.avito.ma/classifieds/images/9757591781?t=images', 'https://content.avito.ma/classifieds/images/7742218029?t=

In [375]:
silver_df = silver_df.drop("attributes_map")

In [377]:
# Print the schema of the silver_df DataFrame
silver_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price_value: double (nullable = true)
 |-- description: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- published_date_text: string (nullable = true)
 |-- image_urls: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- type: string (nullable = true)
 |-- category_type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- site: string (nullable = false)
 |-- Surface habitable: string (nullable = true)
 |-- Caution: string (nullable = true)
 |-- Zoning: string (nullable = true)
 |-- Type d'appartement: string (nullable = true)
 |-- Standing: string (nullable = true)
 |-- Surface totale: string (

In [388]:
silver_df.select('seller_name').show(10, truncate=False)

+------------------------------+
|seller_name                   |
+------------------------------+
|Groupe Ikamati                |
|Groupe Ikamati                |
|Groupe Ikamati                |
|ABAMNY Immobilier             |
|Holiday Resorts Taghazout     |
|Immo continental              |
|GOLF BUILDING                 |
|Immo continental              |
|Groupe Ikamati                |
|Cabinet Immobilier Saïd BENNIS|
+------------------------------+
only showing top 10 rows



In [395]:
silver_df.groupBy('seller_name').count().orderBy('count', ascending=False).show(5, truncate=False)

+----------------------------+-----+
|seller_name                 |count|
+----------------------------+-----+
|agence immobilier mostagimmo|6    |
|safin immobilier            |4    |
|groupe ikamati              |4    |
|immo continental            |3    |
|chawki                      |2    |
+----------------------------+-----+
only showing top 5 rows



In [396]:
from pyspark.sql.functions import lower

# Convert 'seller_name' to lowercase
silver_df = silver_df.withColumn('seller_name', lower(col('seller_name')))

# Show the updated schema and preview the data
silver_df.groupBy('seller_name').count().orderBy('count', ascending=False).show(5, truncate=False)

+----------------------------+-----+
|seller_name                 |count|
+----------------------------+-----+
|agence immobilier mostagimmo|6    |
|safin immobilier            |4    |
|groupe ikamati              |4    |
|immo continental            |3    |
|chawki                      |2    |
+----------------------------+-----+
only showing top 5 rows

